In [3]:
import kivy

In [ ]:
import kivy 
kivy.require('1.10.0') 
   
from kivy.app import App 
from kivy.uix.button import Label 

from kivy.core.window import Window
Window.fullscreen = True

# Inherit Kivy's App class which represents the window 
# for our widgets 
# HelloKivy inherits all the fields and methods 
# from Kivy 
class HelloKivy(App): 
   
    # This returns the content we want in the window 
    def build(self): 
   
        # Return a label widget with Hello Kivy 
        return Label(text ="Hello Geeks")

helloKivy = HelloKivy() 
helloKivy.run()

In [2]:
# main.py 
# import the kivy module 
import kivy  
  
# It’s required that the base Class  
# of your App inherits from the App class. 
from kivy.app import App 
from kivy.uix.gridlayout import GridLayout 
  
# This class stores the info of .kv file 
# when it is called goes to my.kv file 
class MainWidget(GridLayout):  
    pass
  
# we are defining the Base Class of our Kivy App 
class myApp(App): 
    def build(self): 
        # return a MainWidget() as a root widget 
        return MainWidget() 
    
# Here the class MyApp is initialized 
# and its run() method called. 
myApp().run() 

In [6]:
import kivy
from kivy.app import App
from kivy.uix.widget import Widget
from kivy.properties import ObjectProperty
from kivy.uix.label import Label
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.popup import Popup
from kivy.uix.vkeyboard import VKeyboard 


from kivy.core.window import Window
Window.fullscreen = False


class Widgets(Widget):
    player = VKeyboard() 
    def btn(self):
        show_popup()

class P(FloatLayout):
    pass


class MyApp(App):
    def build(self):
        return Widgets()


def show_popup():
    show = P()

    popupWindow = Popup(title="Popup Window", content=show, size_hint=(None,None),size=(400,400))

    popupWindow.open()


if __name__ == "__main__":
    MyApp().run()

In [ ]:
import kivy   
from kivy.core.window import Window
Window.fullscreen = False
# this restricts the kivy version i.e   
# below this kivy version you cannot   
# use the app or software   
      
# base Class of your App inherits from the App class.   
# app:always refers to the instance of your application   
from kivy.app import App 
  
# VKeyboard is an onscreen keyboard 
# for Kivy. Its operation is intended 
# to be transparent to the user.  
from kivy.uix.vkeyboard import VKeyboard 
  
# Create the vkeyboard 
class Test(VKeyboard): 
    player = VKeyboard() 
  
# Create the App class 
class VkeyboardApp(App): 
    def build(self): 
        return Test() 
  
# run the App 
if __name__ == '__main__': 
    VkeyboardApp().run() 

[INFO   ] [Logger      ] Record log in C:\Users\Giulian\.kivy\logs\kivy_20-03-16_12.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.gstreamer" 0.2.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.2.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.2.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.2.0
[INFO   ] [Kivy        ] v1.11.1
[INFO   ] [Kivy        ] Installed at "C:\Users\Giulian\anaconda3\envs\danzeff-OSK-Windows-Remake\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "C:\Users\Giulian\anaconda3\envs\danzeff-OSK-Windows-Remake\python.exe"
[INFO   ] [Factory     ] 184 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_gif (img_pil, img_ffpyplayer ignored)
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INF

In [ ]:
# main.py 
# import the kivy module 
import kivy  
import keyboard
import time
from KivyOnTop import register_topmost
from kivy.graphics import Color, Rectangle
from kivy.lang import Builder


  
# It’s required that the base Class  
# of your App inherits from the App class. 
from kivy.app import App 
from kivy.uix.gridlayout import GridLayout
from kivy.uix.button import Button
from kivy.uix.label import Label 
from kivy.uix.boxlayout import BoxLayout

from kivy.core.window import Window
Window.fullscreen = False


Builder.load_string('''
<MyBoxLayout>
    canvas.before:
        Color:
            rgba: 0.8, 0.9, 0.9, 1
        Rectangle:
            pos: self.pos
            size: self.size
        Color:
            rgba: .5, .5, .5, 1
        Line:
            width: 2
            rectangle: self.x, self.y, self.width, self.height

<TargetBoxLayout>
    canvas.before:
        Color:
            rgba: 1, 0.6, 0.1, 1
        Rectangle:
            pos: self.pos
            size: self.size
        Color:
            rgba: .5, .5, .5, 1
        Line:
            width: 2
            rectangle: self.x, self.y, self.width, self.height
''')
  
# This class stores the info of .kv file 
# when it is called goes to my.kv file 
class MainWidget(GridLayout):  
    pass

class MyBoxLayout(BoxLayout):  
    pass
class TargetBoxLayout(BoxLayout):  
    pass
  
# we are defining the Base Class of our Kivy App 
btn1 = Button(text='Hello 1')

def call2(a):
    #keyboard.write('ciao')
    pass
    
class myApp(App): 
    def on_start(self, *args):
        TITLE = 'WhatAKeyboard'
        Window.set_title(TITLE)
        register_topmost(Window, TITLE)
        
        
            
        keyboard.on_press_key('t', call2 )
    
    def build(self): 
        # return a MainWidget() as a root widget 
        layout = GridLayout(cols=3, row_default_height=60)
        def call1(a, b):
            keyboard.press_and_release('alt + tab')
            time.sleep(1)
            keyboard.write('ciao')


            
        letters_grids = [ 
            [',','a','c','b'], 
            ["'",'d', 'f', 'e'],
            ['!', "g", 'i', 'h'],
            ['-', 'j', 'l', 'k'],
            ['del', 'm', 'n', 'space'],
            ['?', 'o', 'q', 'p'],
            ['(', 'r', 't', 's'],
            [':', 'u', 'w', 'v'],
            [')', 'x', 'z', 'y']
        ]
        
        grid_count = 0
        for grid in letters_grids:
            
            subLayout = GridLayout(cols=3, row_default_height=20)
            for letter in grid:
                subLayout.add_widget(Label(text=''))
                subLayout.add_widget(Label(text=letter))
            subLayout.add_widget(Label(text=''))
            
                
            if grid_count == 4:
                boxLayout = TargetBoxLayout()
            else:
                boxLayout = MyBoxLayout()
            boxLayout.add_widget(subLayout)
            #boxLayour.
            layout.add_widget(boxLayout)
            grid_count = grid_count + 1
            
        return layout

keyboard.on_press_key('j', call2 )
# Here the class MyApp is initialized 
# and its run() method called. 
myApp().run() 